# 4. Dim_reduction

In [1]:
import time
import os

project_name = 'SceneClassification'
step_name = 'Dim_reduction'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

cwd = os.getcwd()
model_path = os.path.join(cwd, 'model')
print('model_path: ' + model_path)

run_name: SceneClassification_Dim_reduction_20171130_211218
model_path: E:\AIChallenger\SceneClassification2017\model


In [2]:
import h5py
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
np.random.seed(2017)

x_train = []
y_train = {}
x_val = []
y_val = {}
x_test = []

cwd = os.getcwd()
feature_cgg16 = os.path.join(cwd, 'model', 'feature_VGG16_{}.h5'.format(171023))
feature_cgg19 = os.path.join(cwd, 'model', 'feature_VGG19_{}.h5'.format(171023))
feature_resnet50 = os.path.join(cwd, 'model', 'feature_ResNet50_{}.h5'.format(171023))
feature_mobilenet = os.path.join(cwd, 'model', 'feature_MobileNet_{}.h5'.format(171023))
feature_xception = os.path.join(cwd, 'model', 'feature_Xception_{}.h5'.format(171023))
feature_inception = os.path.join(cwd, 'model', 'feature_InceptionV3_{}.h5'.format(171127))
feature_resnetv2 = os.path.join(cwd, 'model', 'feature_InceptionResNetV2_{}.h5'.format(171127))
for filename in [feature_inception, feature_resnetv2]:
    with h5py.File(filename, 'r') as h:
        x_train.append(np.array(h['train']))
        y_train = np.array(h['train_label'])
        x_val.append(np.array(h['val']))
        y_val = np.array(h['val_label'])
        x_test.append(np.array(h['test']))

# print(x_train[0].shape)
x_train = np.concatenate(x_train, axis=-1)
# y_train = np.concatenate(y_train, axis=0)
x_val = np.concatenate(x_val, axis=-1)
# y_val = np.concatenate(y_val, axis=0)
x_test = np.concatenate(x_test, axis=-1)
print(x_train.shape)
# print(x_train.shape[1:])
print(len(y_train))
print(x_val.shape)
print(len(y_val))
print(x_test.shape)

(53879, 3584)
53879
(7120, 3584)
7120
(7040, 3584)


In [3]:
%%time
from sklearn.decomposition import IncrementalPCA
from sklearn.externals import joblib
n_comp = 512
ipca = IncrementalPCA(n_components=n_comp)
ipca.fit(x_train)
pca_x_train = ipca.transform(x_train)
pca_x_val = ipca.transform(x_val)
pca_x_test = ipca.transform(x_test)
print(pca_x_train.shape)
print(pca_x_val.shape)
print(pca_x_test.shape)

(53879, 512)
(7120, 512)
(7040, 512)
Wall time: 11min 30s


In [4]:
%%time
#save
pca_model_file_name = run_name + '_pca.pkl'
pca_model_path = os.path.join(model_path, pca_model_file_name)
print(pca_model_path)
joblib.dump(ipca, pca_model_path)
#restore
ipca=joblib.load(pca_model_path)
pca_x_train = ipca.transform(x_train)
pca_x_val = ipca.transform(x_val)
pca_x_test = ipca.transform(x_test)
print(pca_x_train.shape)
print(pca_x_val.shape)
print(pca_x_test.shape)

E:\AIChallenger\SceneClassification2017\model\SceneClassification_Dim_reduction_20171130_211218_pca.pkl
(53879, 512)
(7120, 512)
(7040, 512)
Wall time: 16 s


In [5]:
file_name = os.path.join(cwd, 'model', 'feature_pca_{0}.h5'.format(run_name))
print(file_name)
if os.path.exists(file_name):
    os.remove(file_name)
with h5py.File(file_name) as h:
    h.create_dataset("train", data=pca_x_train)
    h.create_dataset("train_label", data=y_train)
    h.create_dataset("val", data=pca_x_val)
    h.create_dataset("val_label", data=y_val)
    h.create_dataset("test_a", data=pca_x_test)

E:\AIChallenger\SceneClassification2017\model\feature_pca_SceneClassification_Dim_reduction_20171130_211218.h5


In [6]:
print('Done!')

Done!
